# Creating new collections

We've seen collections like lists, strings and tuples that allow indexed access 

   `mylist[0]`

And we've seen collections like dict and set that allow keyed access

`menu_prices_dict['hamburger'] = 59`

In this chapter we will learn how the magic behind collections and access works
and how to create our own containers or customize existing containers

  

## Protocols
Python 'understands' and acts on types in a manner that's called 'duck typing':
  
>   if it walks like a duck <br>
>   and quacks like a duck <br>
>   it is a duck
  
That is, if a type has a certain method (or certain sets of method) then python
will assume that the type provides certain functionality

the method or group of methods needed to fully conform to a usage pattern is called a "protocol".
protocols exists mostly informally, as part of documentation.
there is no need for any 'special' code to conform to a protocol other than implementing the required methods

> conversely in statically-typed languages, such as C++ / Java / C# instead of an informal protocol
> a class often need to inherit from an interface 

### Duck protocol example
Below we give an example of a made up 'Duck protocol': types that have both a walk() and quack() method will be said are 'duck like' or more formally can be said 'conform to the duck protocol'



In [3]:
class Duck:
    def walk(self):
        print('duck walking here')
        
    def quack(self):
        print('duck quacks')
        
# a mallard is a bird that is very similar to a duck
class Mallard:
    def walk(self):
        print('mallard walking here')
        
    def quack(self):
        print('mallard quacks')

class Elephant:
    def walk(self):
        print('elephant walk')
        

animals = [Duck(), Mallard(), Elephant()]
for animal in animals:
    try:
        animal.walk()
        animal.quack()
        print(animal, "is a duck\n")
    except:
        print(animal, ' is not a duck')


duck walking here
duck quacks
<__main__.Duck object at 0x000001B998C404E0> is a duck

mallard walking here
mallard quacks
<__main__.Mallard object at 0x000001B998C40518> is a duck

elephant walk
<__main__.Elephant object at 0x000001B998C402E8>  is not a duck


# Iterator protocol 

Lets start with one of the simplest python protocols: the __iterator__ protocol.

## [What are iterators in Python?][1]
Iterators are everywhere in Python. They are elegantly implemented within for loops, comprehensions, generators etc. but hidden in plain sight.

Iterator in Python is simply an object that can be iterated upon. An object which will return data, one element at a time.

Technically speaking, Python iterator object must implement two special methods, `__iter__()` and `__next__()`, collectively called the iterator protocol.

An object is called iterable if we can get an iterator from it. Most of built-in containers in Python like: list, tuple, string etc. are iterables.

The iter() function (which in turn calls the `__iter__()` method) returns an iterator from them.

## [Iterator Types][2]
Python supports a concept of iteration over containers. This is implemented using two distinct methods; these are used to allow user-defined classes to support iteration. Sequences, described below in more detail, always support the iteration methods.

One method needs to be defined for container objects to provide iteration support:

`container.__iter__()`
> Return an iterator object. The object is required to support the iterator protocol described below. If a container supports different types of iteration, additional methods can be provided to specifically request iterators for those iteration types. (An example of an object supporting multiple forms of iteration would be a tree structure which supports both breadth-first and depth-first traversal.) This method corresponds to the tp_iter slot of the type structure for Python objects in the Python/C API.

The iterator objects themselves are required to support the following two methods, which together form the iterator protocol:

`iterator.__iter__()`
> Return the iterator object itself. This is required to allow both containers and iterators to be used with the for and in statements. This method corresponds to the tp_iter slot of the type structure for Python objects in the Python/C API.

`iterator.__next__()`
> Return the next item from the container. If there are no further items, raise the StopIteration exception. This method corresponds to the tp_iternext slot of the type structure for Python objects in the Python/C API.

Python defines several iterator objects to support iteration over general and specific sequence types, dictionaries, and other more specialized forms. The specific types are not important beyond their implementation of the iterator protocol.

Once an iterator’s __next__() method raises StopIteration, it must continue to do so on subsequent calls. Implementations that do not obey this property are deemed broken.

[1]: https://www.programiz.com/python-programming/iterator
[2]: https://docs.python.org/3.7/library/stdtypes.html#iterator-types

## Example
Below we show an example class that implements the iterator protocol

In [11]:
class VersionedObject :
    """
    VersionedObject is a type that remembers all the past values it held
    and can return the history of its values with a for loop
    """
    def __init__(self, value=None):
        self.__values = [ value ]
        
    def update(self, value):
        self.__values.append(value)

    def latest(self):
        return self.__values[-1]
    
    def __iter__(self):
        return self.Iterator(self.__values)
    
    class Iterator:
        def __init__(self, values):
            self.__index = 0
            self.__values = values
            
        def __next__(self):
            # Return the next item from the container. 
            # If there are no further items, raise the StopIteration exception
            if self.__index is None or len(self.__values) <= self.__index:
                # Once an iterator’s next() method raises StopIteration, it must continue to do so
                self.__index = None 
                raise StopIteration()
            
            value = self.__values[self.__index]
            self.__index += 1
            return value
        
        def __iter__(self):
            # Return the iterator object itself. 
            # This is required to allow both containers and iterators to be used with the for and in statements
            return self #
    
x = VersionedObject([1])
x.update(2)
x.update("third version")
x.update(4)
for older_value in x: # calls the __iter__ method on x
    print(older_value) # calls the __next__ method on the iterator


[1]
2
third version
4


In [13]:
"""

Lets simplify the code for VersionObject, by using the fact that lists []
also support the iterator protocol themselves

"""

class VersionedObject_2 :
    def __init__(self, value=None):
        self.__values = [ value ]
        
    def update(self, value):
        self.__values.append(value)

    def latest(self):
        return self.__values[-1]
    
    def __iter__(self):
        return iter(self.__values) # calls self.__values iterator
        
x = VersionedObject_2([1])
x.update(2)
x.update("third version")
x.update(4)
for older_value in x: # calls the __iter__ method on x
    print(older_value) # calls the __next__ method on the iterator


[1]
2
third version
4


## Sequence protocol

There are three basic sequence types: lists, tuples, and range objects.

Sequences support the following operations:


```
Operation      Result

x in s           True if an item of s is equal to x, else False
x not in s       False if an item of s is equal to x, else True
s + t            the concatenation of s and t
s * n or n * s   equivalent to adding s to itself n times
```

In [53]:
import collections.abc
import math
class MyRange(collections.abc.Sequence):
    def __init__(self, start, stop, step=1):
        self.__start = start
        self.__step = step
        self.__n = max(0, math.ceil((stop-start) / step))
        super().__init__()
        
    def __len__(self):
        return self.__n
    
    def __getitem__(self, offset):
        if self.__n <= offset:
            raise IndexError('range object index out of range')
            
        return self.__start + offset * self.__step

MyRange(0, 1, 2)[0]


0

In [28]:
range(10)[10]

IndexError: range object index out of range

In [24]:
help(range)

Help on class range in module builtins:

class range(object)
 |  range(stop) -> range object
 |  range(start, stop[, step]) -> range object
 |  
 |  Return an object that produces a sequence of integers from start (inclusive)
 |  to stop (exclusive) by step.  range(i, j) produces i, i+1, i+2, ..., j-1.
 |  start defaults to 0, and stop is omitted!  range(4) produces 0, 1, 2, 3.
 |  These are exactly the valid indices for a list of 4 elements.
 |  When step is given, it specifies the increment (or decrement).
 |  
 |  Methods defined here:
 |  
 |  __bool__(self, /)
 |      self != 0
 |  
 |  __contains__(self, key, /)
 |      Return key in self.
 |  
 |  __eq__(self, value, /)
 |      Return self==value.
 |  
 |  __ge__(self, value, /)
 |      Return self>=value.
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __getitem__(self, key, /)
 |      Return self[key].
 |  
 |  __gt__(self, value, /)
 |      Return self>value.
 |  
 |  __hash__(self, /)
 |

In [38]:
help(collections.abc.Sequence)

Help on class Sequence in module collections.abc:

class Sequence(Reversible, Collection)
 |  All the operations on a read-only sequence.
 |  
 |  Concrete subclasses must override __new__ or __init__,
 |  __getitem__, and __len__.
 |  
 |  Method resolution order:
 |      Sequence
 |      Reversible
 |      Collection
 |      Sized
 |      Iterable
 |      Container
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __contains__(self, value)
 |  
 |  __getitem__(self, index)
 |  
 |  __iter__(self)
 |  
 |  __reversed__(self)
 |  
 |  count(self, value)
 |      S.count(value) -> integer -- return number of occurrences of value
 |  
 |  index(self, value, start=0, stop=None)
 |      S.index(value, [start, [stop]]) -> integer -- return first index of value.
 |      Raises ValueError if the value is not present.
 |      
 |      Supporting start and stop arguments is optional, but
 |      recommended.
 |  
 |  -----------------------------------------------------------------